#### Objective: Use race migration table to generate race migration points

In [1]:
import pandas as pd
import geopandas as gpd
import ast,os,random
pd.set_option('display.float_format','{:.1f}'.format)
import warnings
warnings.filterwarnings('ignore')
import cudf, cupy as cp
import numpy as np
import time
import math
import sys,os,datetime,random
from shapely.geometry import Point
# pd.set_option('display.max_colwidth', -1)

#### Load data

In [8]:
df = cudf.read_csv('data/total_population_gen_df.csv').drop('Unnamed: 0',axis=1)
df.head()

,ID20,STATE,points
0,10010201001000,1,52.0
1,10010201001001,1,34.0
2,10010201001002,1,81.0
3,10010201001003,1,17.0
4,10010201001005,1,8.0


In [3]:
# df = df[df.STATE==6]
# len(df)//3
# df= df.iloc[:len(df)//3]

In [10]:
print(len(df))

161904


In [2]:
def random_points_in_polygon(number, polygon):
    # print(polygon)
    points_x = np.array([])
    points_y = np.array([])
    min_x, min_y, max_x, max_y = polygon.bounds
    i= 0
    while i < number:
        point_x = random.uniform(min_x, max_x)
        point_y = random.uniform(min_y, max_y)
        if polygon.contains(Point(point_x, point_y)):
            points_x = np.append(points_x, point_x)
            points_y = np.append(points_y, point_y)
            i += 1
    return points_x, points_y # returns list of points(lat), list of points(long)
def generate_data(state, df_temp, gpdf):
    t1 = datetime.datetime.now()
    geoid_index_df = df_temp.index.to_numpy()
    final_points_x = np.array([])
    final_points_y = np.array([])
    geoid = np.array([])
    # Add additional features
    county = np.array([])
    p_delta = np.array([])
    p_net = np.array([])
    
    
    f=0
    for index, row in gpdf.iterrows():
        f+=1
        points_x = np.array([])
        points_y = np.array([])
        geoid_temp = np.array([])
        
        if row['GEOID20'] in geoid_index_df:
            num_points = df_temp.loc[row['GEOID20']]
            polygon = row['geometry']
            #print(row['GEOID10'])
            #print('SUCCESS')
            num_points = df_temp.loc[row['GEOID20']] # store population
            polygon = row['geometry']

      
            if polygon is not None:
                points_x, points_y = random_points_in_polygon(num_points, polygon)
                # print(points_x,points_y)
                geoid_temp = np.array([row['GEOID20']]*len(points_x))
                geoid = np.append(geoid,geoid_temp)
                final_points_x = np.append(final_points_x, points_x)
                # print(final_points_x)
                final_points_y = np.append(final_points_y, points_y)
                print('Processing '+str(state)+' - Completed:', "{0:0.2f}".format((index/len(gpdf))*100), '%', end='')
                print('', end='\r')
                
            # if f==11:
            #     break

    print('Processing for '+str(state)+' complete \n total time', datetime.datetime.now() - t1)
    df_fin = cudf.DataFrame({'GEOID20': geoid,'x': final_points_x, 'y':final_points_y}) #,'COUNTY':county,'p_delta':p_delta,'p_net':p_net})
    df_fin.GEOID20 = df_fin.GEOID20[1:].astype('int').astype('str')
    df_fin.GEOID20 = df_fin.GEOID20.fillna(method='bfill')
    
    df_fin.to_csv('data/total_population/population_%s_1'%str(state)+'.csv', index=False)
def exec_data(state_key_list):
    c=0
    for i in state_key_list:
        print(i)
        c+=1
        if i< 10:
            i_str = '0'+str(i)
        else:
            i_str = str(i)
        # path = 'census_2020_data/nhgis0003_shape/nhgis0003_shapefile_tl2020_%s0_block_2020/%s_block_2020.shp'%(i_str,states[i])
        path ='data/tl_shapefiles/tl_2021_%s_tabblock20.shp'%(i_str)
        #print(path)
        print("started reading shape file for state ", states[i])
        if os.path.isfile(path):    
            gpdf = gpd.read_file(path)[['GEOID20', 'geometry']].sort_values('GEOID20').reset_index(drop=True)
            gpdf.GEOID20 = gpdf.GEOID20.astype('int64')
            gpdf = gpdf[(gpdf.GEOID20>=480019501001000) & (gpdf.GEOID20<=481439502032029)].reset_index(drop=True)
            print("completed reading shape file for state ", states[i])
            df_temp = df.query('STATE == @i')[['ID20', 'points']]
            df_temp.index = df_temp.ID20
            df_temp = df_temp['points']
            # print(gpdf.head(3))
            # print(df_temp)
            print("starting to generate data for "+str(states[i])+"... ")
            generate_data(states[i], df_temp, gpdf)
            del(df_temp)
        else:
            print("shape file does not exist")
            continue
        # if c==2:
        #     break 

In [3]:
# states = {1 :"AL",2 :"AK",4 :"AZ",5 :"AR",6 :"CA",8 :"CO",9 :"CT",10:"DE",11:"DC",12:"FL",13:"GA",15:"HI",
#           16:"ID",17:"IL",18:"IN",19:"IA",20:"KS",21:"KY",22:"LA",23:"ME",24:"MD",25:"MA",26:"MI",27:"MN",
#           28:"MS",29:"MO",30:"MT",31:"NE",32:"NV",33:"NH",34:"NJ",35:"NM",36:"NY",37:"NC",38:"ND",39:"OH",
#           40:"OK",41:"OR",42:"PA",44:"RI",45:"SC",46:"SD",47:"TN",48:"TX",49:"UT",50:"VT",51:"VA",53:"WA",
#           54:"WV",55:"WI",56:"WY",72:"PR"}
# states = {6:"CA"}

In [13]:
exec_data(states.keys())

48
started reading shape file for state  TX
completed reading shape file for state  TX
starting to generate data for TX... 
Processing for TX complete 100.00 %
 total time 3:08:48.306832


### Concat Parts

In [2]:
def merge_parts(state_key_list):
    concat_states = cudf.DataFrame()
    c=0
    for i in state_key_list:
        for c in range(1,4):
            if i< 10:
                i_str = '0'+str(i)
            else:
                i_str = str(i)
            path = 'data/total_population/population_%s_%s'%(str(states[i]),c)+'.csv'
            # print(path)
            if os.path.isfile(path):    
                temp = cudf.read_csv(path) # Load shape files
                concat_states = cudf.concat([concat_states,temp])
            else:
                print("population file does not exist")
                continue
        return concat_states

In [4]:
concat_parts = merge_parts(states)

In [5]:
concat_parts =concat_parts.reset_index(drop=True)
concat_parts

,GEOID20,x,y
0,60014001001001,-122.2,37.9
1,60014001001001,-122.2,37.9
2,60014001001001,-122.2,37.9
3,60014001001001,-122.2,37.9
4,60014001001001,-122.2,37.9
...,...,...,...
59325523,61150411021048,-121.3,39.4
59325524,61150411021048,-121.3,39.4
59325525,61150411021048,-121.3,39.4
59325526,61150411021048,-121.3,39.4


In [9]:
df[df.STATE==48].points.sum()

42742567.0

In [21]:
df.points.sum()

59325528.0

In [10]:
concat_parts.to_pandas().to_csv('data/total_population/population_CA')

### Concat States

In [2]:
def merge_shape_and_states(state_key_list):
    concat_states = cudf.DataFrame()
    
    for i in state_key_list:
        if i< 10:
            i_str = '0'+str(i)
        else:
            i_str = str(i)
        path = 'data/total_population/population_%s'%str(states[i])+'.csv'
        if os.path.isfile(path):    
            temp = cudf.read_csv(path) # Load shape files
            concat_states = cudf.concat([concat_states,temp])
        else:
            print(i)
            print("population file does not exist")
            continue
        print(i)
    return concat_states

In [3]:
# states = {1 :"AL",2 :"AK",4 :"AZ",5 :"AR",6 :"CA",8 :"CO",9 :"CT",10:"DE",11:"DC",12:"FL",13:"GA",15:"HI",
#           16:"ID",17:"IL",18:"IN",19:"IA",20:"KS",21:"KY",22:"LA",23:"ME",24:"MD",25:"MA",26:"MI",27:"MN",
#           28:"MS",29:"MO",30:"MT",31:"NE",32:"NV",33:"NH",34:"NJ",35:"NM",36:"NY",37:"NC",38:"ND",39:"OH",
#           40:"OK",41:"OR",42:"PA",44:"RI",45:"SC",46:"SD",47:"TN",48:"TX",49:"UT",50:"VT",51:"VA",53:"WA",
#           54:"WV",55:"WI",56:"WY",72:"PR"}
states = {1 :"AL",2 :"AK",4 :"AZ",5 :"AR",6 :"CA",8 :"CO",9 :"CT",10:"DE",11:"DC",12:"FL",13:"GA",15:"HI",
          16:"ID",17:"IL",18:"IN",19:"IA",20:"KS",21:"KY",22:"LA",23:"ME",24:"MD",25:"MA",26:"MI",27:"MN",
          28:"MS"} # part1
states = {29:"MO",30:"MT",31:"NE",32:"NV",33:"NH",34:"NJ",35:"NM",36:"NY",37:"NC",38:"ND",39:"OH",
          40:"OK",41:"OR",42:"PA",44:"RI",45:"SC",46:"SD",47:"TN",48:"TX",49:"UT",50:"VT",51:"VA",53:"WA",
          54:"WV",55:"WI",56:"WY",72:"PR"} #part2

In [4]:
indv_df = merge_shape_and_states(states.keys()).drop('Unnamed: 0',axis=1)
indv_df.rename(columns={'GEOID20':'ID20'},inplace=True)
indv_df.head()

29
30
31
32
33
34
35
36
37
38
39
40
41
42
44
45
46
47
48
49
50
51
53
54
55
56
72


,ID20,x,y
0,290019501001000,-92.4,40.3
1,290019501001000,-92.4,40.3
2,290019501001001,-92.4,40.3
3,290019501001001,-92.4,40.3
4,290019501001001,-92.4,40.3


In [5]:
len(indv_df)

248001113

In [ ]:
# indv_df.to_pandas().to_parquet('data/total_part1.parquet')

In [6]:
# indv_df.to_pandas().to_parquet('data/total_part2.parquet')

### Use processed dfs

In [2]:
# df1 = pd.read_parquet('data/total_part1.parquet')
# df2 = pd.read_parquet('data/total_part2.parquet')

In [3]:
# merged = pd.concat([df1,df2])

In [5]:
# len(merged)

504475979

In [6]:
# gpu = cudf.from_pandas(merged)

In [9]:
# merged.to_parquet('data/total_parts_combined.parquet')

In [ ]:
# dataset = indv_df.merge(df,on='ID20',how='left').sort_values('ID20')
# dataset.head()